In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import os
import datatable
from sklearn.model_selection import train_test_split as sk_train_test_split


path = '/kaggle/input/jane-street-market-prediction'

train = datatable.fread(os.path.join(path,"train.csv")).to_pandas()
test = datatable.fread(os.path.join(path,"example_test.csv")).to_pandas()
submission = pd.read_csv(os.path.join(path,'example_sample_submission.csv'))

train = train.astype({c_type: np.float32 for c_type in train.select_dtypes(include='float64').columns})

In [3]:
features = [f for f in train.columns if not f.find('feature')]
train['action']=(train['resp']>0)*1
train.head()
train = train.fillna(train.mean())
train = train[train['weight']!=0]

In [4]:
train_x, val_test_x, train_y, val_test_y = sk_train_test_split(train[features+['date']],train.action,test_size=0.33, random_state=42)

In [5]:
lgb_train = lgb.Dataset(train_x,label=train_y)
val_lgb_test = lgb.Dataset(val_test_x,label=val_test_y)

In [6]:
parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': 0,
    'boosting_type': 'gbdt',
    'lambda_l1': 0.17609455161492504,
    'lambda_l2': 0.5037981573325253,
    'num_leaves': 248,
    'feature_fraction': 0.5622921223059042,
    'bagging_fraction': 0.870103740422135,
    'bagging_freq': 7,
    'min_child_samples': 5
    
}
model = lgb.train(
    parameters, lgb_train, valid_sets=[lgb_train,val_lgb_test],valid_names = ['train', 'valid'],
    num_boost_round= 10000,
    verbose_eval= 50,
    early_stopping_rounds= 10
)

Training until validation scores don't improve for 10 rounds
[50]	train's auc: 0.661515	valid's auc: 0.622928
[100]	train's auc: 0.713792	valid's auc: 0.642054
[150]	train's auc: 0.752157	valid's auc: 0.659288
[200]	train's auc: 0.783253	valid's auc: 0.669581
[250]	train's auc: 0.807727	valid's auc: 0.676958
[300]	train's auc: 0.829409	valid's auc: 0.682657
[350]	train's auc: 0.846862	valid's auc: 0.6853
Early stopping, best iteration is:
[340]	train's auc: 0.84357	valid's auc: 0.685467


In [7]:
import janestreet
env = janestreet.make_env()
iter_test = env.iter_test()

In [8]:
%%time

# # predict = np.where(model.predict(test[features]) < 0.5, 0, 1)
for (test_df, sample_prediction_df) in iter_test:
    predict = np.where(model.predict(test_df[features+['date']]) < 0.5, 0, 1)
    
    sample_prediction_df['action'] = predict
    env.predict(sample_prediction_df)


CPU times: user 19min 41s, sys: 30.9 s, total: 20min 12s
Wall time: 5min 16s


In [9]:
os.listdir()

['__notebook__.ipynb', 'submission.csv']